In [2]:
import requests
import pandas as pd

In [3]:
def find_beer_image_url(beer_name, company_name, api_key, cx):
    search_query = f"{beer_name} {company_name} beer"
    params = {
        "q": search_query,
        "cx": cx,
        "key": api_key,
        "searchType": "image",
        "num": 1
    }

    response = requests.get("https://www.googleapis.com/customsearch/v1", params=params)
    response.raise_for_status()
    results = response.json()

    if "items" in results:
        return results["items"][0]["link"]
    else:
        return None


In [26]:
import json

with open('../../credentials_google_search.json', 'r') as f:
    creds = json.load(f)
    #content = f.read()
    #print("FILE CONTENT:")
    #print(content)

api_key = creds['google']['api_key']
cx = creds['google']['cx']

In [27]:


db_beers = pd.read_csv("../../data/clean/beer_profile_and_ratings.csv")


#image_url = find_beer_image_url(beer_name, company_name, api_key, cx)
#print("Image URL:", image_url)


In [28]:
print(db_beers.columns.tolist())
db_beers_names_breweries = db_beers[["name", "brewery"]].drop_duplicates()
num_of_beers =len(db_beers_names_breweries)
print("Number of beers:", num_of_beers)

['name', 'style', 'brewery', 'beer name (full)', 'description', 'abv', 'min ibu', 'max ibu', 'astringency', 'body', 'alcohol', 'bitter', 'sweet', 'sour', 'salty', 'fruits', 'hoppy', 'spices', 'malty', 'review_aroma', 'review_appearance', 'review_palate', 'review_taste', 'review_overall', 'number_of_reviews']
Number of beers: 3197


In [ ]:
import time

for i in range(len(db_beers_names_breweries)):
    beer_name = db_beers_names_breweries.iloc[i]["name"]
    company_name = db_beers_names_breweries.iloc[i]["brewery"]
    print(f"Beer: {beer_name}, Brewery: {company_name}")
    
    try:
        image_url = find_beer_image_url(beer_name, company_name, api_key, cx)
        print("Image URL:", image_url)
        db_beers_names_breweries.at[i, "image_url"] = image_url
    except Exception as e:
        print(f"Error at row {i}: {e}")
        db_beers_names_breweries.at[i, "image_url"] = None
    
    time.sleep(1.5)  # delay between requests


KeyError: 'beer'